In [1]:
import pandas as pd
import seaborn as sns

In [2]:
projects_df = pd.read_csv("./raw_data/Projects.csv")

In [3]:
projects_df

,agencyCode,agencyName,currentUII,investmentName,investmentType,uniqueProjectId,projectId,agencyProjectId,projectName,projectGoal,...,plannedCost,projectedCost,actualCost,scheduleVariance(days),scheduleVariance(%),scheduleVarianceColor,costVariance($M),costVariance(%),costVarianceColor,updatedTime
0,6,Department of Commerce,006-000000136,Census - Enterprise Data Lake (EDL),Major IT Investments,NaN,0001D21002,6037e5372b61a9426215080f,Enterprise Data Lake,"To build a high performance, secure and scalab...",...,NaN,NaN,Array,28,9.4276,green,3.071153,-27.764000,green,2021-09-29 18:06:16
1,6,Department of Commerce,006-000000136,Census - Enterprise Data Lake (EDL),Major IT Investments,NaN,0001P22003,61bbba083e19e8429e168fd3,EDL User Acceptance Testing,To test targeted usage needs to ensure all fea...,...,Array,Array,Array,0,0.0000,green,-0.456794,-10.418407,green,2023-01-30 20:57:56
2,6,Department of Commerce,006-000000136,Census - Enterprise Data Lake (EDL),Major IT Investments,NaN,0001P22004,61bbbfcd3e19e8429e168fe5,Integrate EDL with Major Enterprise Initiatives,Integrate the Enterprise Data Lake into a comm...,...,Array,Array,Array,0,0.0000,green,-2.171520,-20.853606,green,2023-06-27 11:42:08
3,6,Department of Commerce,006-000000136,Census - Enterprise Data Lake (EDL),Major IT Investments,NaN,0001P22006,61bbc5193e19e8429e16900d,Cumulus Integration Platform,Complete buildout of Cumulus in AWS GovCloud\n,...,NaN,NaN,Array,0,0.0000,green,0.112797,-3.937000,green,2022-03-03 16:32:38
4,6,Department of Commerce,006-000000136,Census - Enterprise Data Lake (EDL),Major IT Investments,NaN,0001P22002,61bbab913e19e8429e168fa0,Census Data Lake to Enterprise Data Lake Data ...,Migrate Decennial Census data sets from Census...,...,NaN,NaN,Array,0,0.0000,green,-0.577788,17.570700,yellow,2022-03-03 16:32:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3849,393,National Archives and Records Administration,393-000000087,Citizen Services,Major IT Investments,NaN,21180P,63593b25ae7b9990ac90f001,History Hub Replacement,Migrate data from the existing instance of HH ...,...,Array,Array,Array,0,0.0000,green,0.000000,0.000000,green,2022-11-30 22:07:54
3850,393,National Archives and Records Administration,393-000000087,Citizen Services,Major IT Investments,NaN,16060P,6345ba81be212e3efc85e561,Obama Library Website Upgrade (FOIA Reading Room),Migrate existing Obama library website from Dr...,...,Array,Array,Array,0,0.0000,green,0.000000,0.000000,green,2022-10-27 18:42:54
3851,393,National Archives and Records Administration,393-000000087,Citizen Services,Major IT Investments,NaN,16062J,63593991c715e102f37fc4d0,Carter Redesign and Migration,Included in this work will be a redesign of th...,...,Array,Array,Array,0,0.0000,green,0.000000,0.000000,green,2022-11-30 22:07:54
3852,393,National Archives and Records Administration,393-000000250,IT Security and Compliance,Standard IT Investments,NaN,21507J,634721df320993bba26d612d,HSPD12,Work with each application across NARA to deve...,...,Array,Array,Array,0,0.0000,green,0.000000,0.000000,green,2022-10-27 18:47:18
